# Pandas

In [88]:
import pandas as pd
import pickle

In [89]:
preprocessing = pd.read_csv("./kbo.csv")

In [90]:
with open('./kbo_player.pkl' , 'rb') as f:
    original_dict = pickle.load(f)

## 중복 값 확인하기

In [91]:
preprocessing[preprocessing['선수명'].duplicated()].shape

(40, 15)

In [92]:
original = pd.DataFrame(original_dict)

In [93]:
original_extract = original[['선수명', '등번호', 'Team']].copy()

In [94]:
preprocessing.head(1)

,선수명,등번호,생년월일,포지션,경력,입단 계약금,연봉,지명순위,입단년도,체중,신장,BMI,나이,탄생요일,연봉(만원)
0,강정현,No.100,1995년 09월 23일,투수(우투우타),남부민초-대신중-부경고-원광대-LG-상무,7000만원,3300만원,19 LG 2차 4라운드 35순위,19LG,90,180,27.777778,28,토,3300.0


In [95]:
original_extract.head(1)

,선수명,등번호,Team
0,강매성,No.95,SSG 랜더스


In [96]:
total = pd.merge(preprocessing, original_extract, left_on=['선수명', '등번호'], right_on=['선수명', '등번호'], how='left')

In [97]:
total.shape

(829, 16)

In [98]:
df = total.copy()

## 컬럼별 결측치 확인 

In [99]:
df.isnull().sum()

선수명         0
등번호         0
생년월일        0
포지션         0
경력          0
입단 계약금    116
연봉          2
지명순위        1
입단년도      158
체중          0
신장          0
BMI         0
나이          0
탄생요일        0
연봉(만원)      2
Team        0
dtype: int64

In [100]:
# 결측치는 float 타입
type(df['입단년도'].values[1])

str

## unique한 값 출력

In [101]:
df['포지션'].unique()

array(['투수(우투우타)', '내야수(우투우타)', '포수(우투우타)', '투수(좌투좌타)', '내야수(좌투좌타)',
       '외야수(우투좌타)', '내야수(우투좌타)', '투수(우언우타)', '외야수(좌투좌타)', '투수(우투좌타)',
       '외야수(우투우타)', '포수(우투좌타)', '투수(좌투우타)', '투수(우언좌타)', '투수(우투양타)',
       '외야수(우투양타)', '내야수(우투양타)', '투수(우언양타)'], dtype=object)

In [102]:
df['Team'].unique()

array(['LG 트윈스', 'SSG 랜더스', '삼성 라이온즈', '두산 베어스', '롯데 자이언츠', 'KIA 타이거즈',
       '키움 히어로즈', '고양 히어로즈', 'KT 위즈', 'NC 다이노스', '한화 이글스'], dtype=object)

In [103]:
# Team에서 "고양"이라는 단어를 포함하고 있는 컬럼 출력
df['Team'].str.contains("고양")
# df[df['Team'].str.contains("고양")]

0      False
1      False
2      False
3      False
4      False
       ...  
824    False
825    False
826    False
827    False
828    False
Name: Team, Length: 829, dtype: bool

In [104]:
# 고양 히어로즈 팀 소속 선수들 키움 히어로즈로 변경
df.loc[df['Team'].str.contains("고양"), 'Team'] = '키움 히어로즈'

In [105]:
df['Team'].value_counts()

LG 트윈스      89
두산 베어스      88
KIA 타이거즈    84
KT 위즈       83
NC 다이노스     83
한화 이글스      83
SSG 랜더스     82
삼성 라이온즈     80
키움 히어로즈     79
롯데 자이언츠     78
Name: Team, dtype: int64

## groupby

In [106]:
# df.groupby(['Team'])[['연봉(만원)']].mean().sort_values(by=['연봉(만원)'])
df.groupby(['Team'])[['연봉(만원)']].median().sort_values(by=['연봉(만원)'])

,연봉(만원)
Team,
NC 다이노스,3200.0
SSG 랜더스,3200.0
한화 이글스,3200.0
KT 위즈,3300.0
두산 베어스,3300.0
롯데 자이언츠,3400.0
키움 히어로즈,3500.0
KIA 타이거즈,3600.0
LG 트윈스,4000.0


In [107]:
df.groupby(['Team','포지션'], as_index=False)[['연봉(만원)']].mean()
df.groupby(['Team','포지션'], as_index=False)[['연봉(만원)']].count()

,Team,포지션,연봉(만원)
0,KIA 타이거즈,내야수(우투우타),10
1,KIA 타이거즈,내야수(우투좌타),7
2,KIA 타이거즈,내야수(좌투좌타),1
3,KIA 타이거즈,외야수(우투우타),8
4,KIA 타이거즈,외야수(우투좌타),5
...,...,...,...
114,한화 이글스,투수(우언우타),3
115,한화 이글스,투수(우투우타),26
116,한화 이글스,투수(좌투좌타),12
117,한화 이글스,포수(우투우타),8


In [108]:
df.loc[df['연봉'].notnull(), '용병'] = df.loc[df['연봉'].notnull(), '연봉'].apply(lambda x : x[-2:] == "달러")

In [109]:
df['용병'].value_counts()

False    797
True      30
Name: 용병, dtype: int64

In [110]:
df.groupby(['Team'])[['용병']].sum()

,용병
Team,
KIA 타이거즈,3
KT 위즈,3
LG 트윈스,3
NC 다이노스,3
SSG 랜더스,3
두산 베어스,3
롯데 자이언츠,3
삼성 라이온즈,3
키움 히어로즈,3


## 네이버 증권 데이터 불러오기

In [39]:
import requests

In [46]:
url = "https://finance.naver.com//item/sise_day.naver?code=005930&page=1"
head = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}

In [47]:
r = requests.get(url, headers = head)

In [57]:
stock1 = pd.read_html(r.text)[0].dropna()

In [58]:
stock1

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.04.12,67600.0,300.0,67600.0,67700.0,67400.0,2914475.0
2,2022.04.11,67900.0,100.0,67800.0,68100.0,67400.0,12263735.0
3,2022.04.08,67800.0,200.0,68100.0,68300.0,67700.0,15453191.0
4,2022.04.07,68000.0,500.0,68500.0,68500.0,68000.0,20683327.0
5,2022.04.06,68500.0,700.0,68600.0,68800.0,68500.0,15517308.0
9,2022.04.05,69200.0,100.0,69400.0,69600.0,69100.0,8467248.0
10,2022.04.04,69300.0,200.0,68900.0,69300.0,68600.0,11107905.0
11,2022.04.01,69100.0,500.0,69500.0,69500.0,69000.0,15916846.0
12,2022.03.31,69600.0,300.0,69900.0,70200.0,69600.0,12510366.0
13,2022.03.30,69900.0,300.0,70300.0,70500.0,69800.0,12670187.0


In [53]:
stock2 = pd.read_html(requests.get("https://finance.naver.com//item/sise_day.naver?code=005930&page=2", headers=head).text)[0].dropna()

In [54]:
stock2

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.03.29,70200.0,500.0,70000.0,70300.0,69800.0,13686208.0
2,2022.03.28,69700.0,100.0,69500.0,69900.0,69200.0,12619289.0
3,2022.03.25,69800.0,0.0,70100.0,70200.0,69600.0,12986010.0
4,2022.03.24,69800.0,700.0,69600.0,70300.0,69600.0,37943357.0
5,2022.03.23,70500.0,200.0,70600.0,71200.0,70300.0,12398025.0
9,2022.03.22,70300.0,400.0,69900.0,70500.0,69900.0,9402666.0
10,2022.03.21,69900.0,800.0,70900.0,71000.0,69900.0,11169002.0
11,2022.03.18,70700.0,500.0,70600.0,70900.0,70200.0,14410038.0
12,2022.03.17,71200.0,800.0,71200.0,71800.0,70900.0,17646315.0
13,2022.03.16,70400.0,900.0,70200.0,70500.0,69700.0,10175750.0


In [59]:
pd.concat([stock1, stock2])

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.04.12,67600.0,300.0,67600.0,67700.0,67400.0,2914475.0
2,2022.04.11,67900.0,100.0,67800.0,68100.0,67400.0,12263735.0
3,2022.04.08,67800.0,200.0,68100.0,68300.0,67700.0,15453191.0
4,2022.04.07,68000.0,500.0,68500.0,68500.0,68000.0,20683327.0
5,2022.04.06,68500.0,700.0,68600.0,68800.0,68500.0,15517308.0
9,2022.04.05,69200.0,100.0,69400.0,69600.0,69100.0,8467248.0
10,2022.04.04,69300.0,200.0,68900.0,69300.0,68600.0,11107905.0
11,2022.04.01,69100.0,500.0,69500.0,69500.0,69000.0,15916846.0
12,2022.03.31,69600.0,300.0,69900.0,70200.0,69600.0,12510366.0
13,2022.03.30,69900.0,300.0,70300.0,70500.0,69800.0,12670187.0


## 코스피 종목 주식정보 불러오기

- 거래소 정보데이터 시스템에서 종목정보를 post 방식으로 가져온다.
- 가져온 데이터를 DataFrame으로 저장
- 시장구분을 KOSPI만 추출
- 추출된 종목 대상으로 네이버 금융 페이지에서 각 종목마다 10페이지 가져옴
- 종목코드 파생변수 추가
- 종목별 df의 row 값( a.iloc[::-1] )을 역순으로 변경(1번째 row값이 가장 오래된 날짜)
- 전체 df를 concat() 이용 해서 하나의 df로 생성

In [77]:
krx = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

payload= {"bld" : "dbms/MDC/STAT/standard/MDCSTAT01901",
    "locale" : "ko_KR",
    "mktId" : "ALL",
    "share" : "1",
    "csvxls_isNo" : "false",}

krx_r = requests.post(krx, data=payload)

In [84]:
krx_df = pd.DataFrame(krx_r.json()['OutBlock_1'])

In [87]:
kospi_mask = krx_df['MKT_TP_NM'] == 'KOSPI'
krx_df.loc[kospi_mask, 'ISU_SRT_CD']

In [113]:
krx_df.head()

,ISU_CD,ISU_SRT_CD,ISU_NM,ISU_ABBRV,ISU_ENG_NM,LIST_DD,MKT_TP_NM,SECUGRP_NM,SECT_TP_NM,KIND_STKCERT_TP_NM,PARVAL,LIST_SHRS
3,KR7095570008,095570,AJ네트웍스보통주,AJ네트웍스,"AJ Networks Co.,Ltd.",2015/08/21,KOSPI,주권,,보통주,"1,000","46,822,295"
4,KR7006840003,006840,AK홀딩스보통주,AK홀딩스,"AK Holdings, Inc.",1999/08/11,KOSPI,주권,,보통주,"5,000","13,247,561"
5,KR7282330000,282330,BGF리테일보통주,BGF리테일,BGF Retail,2017/12/08,KOSPI,주권,,보통주,"1,000","17,283,906"
6,KR7027410000,027410,BGF보통주,BGF,BGF,2014/05/19,KOSPI,주권,,보통주,"1,000","95,716,791"
7,KR7138930003,138930,BNK금융지주보통주,BNK금융지주,BNK Financial Group Inc.,2011/03/30,KOSPI,주권,,보통주,"5,000","325,935,246"


In [163]:
def get_stock(code):
    
    temp = []
    for i in range(1, 11):
        temp.append(pd.read_html(requests.get("https://finance.naver.com//item/sise_day.naver?code={}&page={}".format(code,i), headers=head).text)[0].dropna())
    
    stock = pd.concat(temp)
    stock['종목코드'] = code
    
    return stock

In [169]:
temp2=[]
for x in krx_df['ISU_SRT_CD']:
    temp2.append(get_stock(x).iloc[::-1])
    print(x)

095570
006840
282330
027410
138930
001465
001460
001045
00104K
001040
011155
011150
000590
012030
016610
005830
000995
000990
139130
001530
000215
001880
000210
37550K
375500
155660
069730
017940
365550
383220
007700
078935
006360
078930
012630
294870
097230
003560
175330
234080
001065
001067
001060
096760
105560
009070
016385
016380
001390
033180
001940
025000
092230
000040
093050
003555
034220
003550
051905
051900
373220
032640
011070
066575
066570
051915
051910
079550
010120
000680
006260
001120
023150
035420
181710
338100
034310
008260
004255
004250
010060
010955
010950
005090
001380
001770
002360
009160
123700
025530
03473K
011790
018670
001745
001740
210980
395400
034730
402340
361610
096775
096770
001515
001510
28513K
285130
017670
064960
100840
003570
036530
005610
011810
077970
071970
084870
002710
024070
000500
000860
035250
011420
002100
009450
267290
012320
000050
214390
012610
009140
013580
012205
012200
002140
010130
002240
009290
017040
017900
037710
030610
339770
007690

In [170]:
stock_total = pd.concat(temp2)

In [171]:
stock_total

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드
13,2021.11.16,5930.0,80.0,5900.0,5980.0,5850.0,130598.0,095570
12,2021.11.17,5820.0,110.0,5930.0,5940.0,5730.0,132754.0,095570
11,2021.11.18,5830.0,10.0,5820.0,5890.0,5750.0,143015.0,095570
10,2021.11.19,5840.0,10.0,5740.0,5900.0,5720.0,54112.0,095570
9,2021.11.22,6040.0,200.0,5840.0,6110.0,5840.0,244193.0,095570
...,...,...,...,...,...,...,...,...
5,2022.04.06,3095.0,85.0,3205.0,3260.0,2990.0,2719696.0,003280
4,2022.04.07,2945.0,150.0,3070.0,3075.0,2940.0,965012.0,003280
3,2022.04.08,3090.0,145.0,2955.0,3155.0,2890.0,2628593.0,003280
2,2022.04.11,3020.0,70.0,3115.0,3170.0,3000.0,1004549.0,003280


In [175]:
stock_total.value_counts("종목코드")

종목코드
000020    100
036580    100
03473K    100
034830    100
035000    100
         ... 
396690     94
373220     94
402340     91
404990     84
417310     28
Length: 939, dtype: int64

## 중복 값 확인

In [176]:
stock_total[stock_total.duplicated()]

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드
12,2022.01.27,505000.0,92000.0,597000.0,598000.0,450000.0,15946992.0,373220
11,2022.01.28,450000.0,55000.0,476000.0,483000.0,445000.0,4559773.0,373220
10,2022.02.03,477000.0,27000.0,458000.0,495500.0,441000.0,2918435.0,373220
9,2022.02.04,504000.0,27000.0,476500.0,505000.0,476000.0,2088996.0,373220
5,2022.02.07,548000.0,44000.0,520000.0,548000.0,511000.0,1911176.0,373220
...,...,...,...,...,...,...,...,...
1,2022.03.29,5420.0,10.0,5440.0,5460.0,5400.0,693894.0,417310
2,2022.03.28,5410.0,150.0,5260.0,5490.0,5260.0,9086883.0,417310
1,2022.03.29,5420.0,10.0,5440.0,5460.0,5400.0,693894.0,417310
2,2022.03.28,5410.0,150.0,5260.0,5490.0,5260.0,9086883.0,417310


## 중복 값 제거

In [178]:
stock_total.drop_duplicates(inplace = True)

## Index 초기화

In [182]:
stock_total.reset_index(drop=True, inplace = True)

In [183]:
stock_total

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드
0,2021.11.16,5930.0,80.0,5900.0,5980.0,5850.0,130598.0,095570
1,2021.11.17,5820.0,110.0,5930.0,5940.0,5730.0,132754.0,095570
2,2021.11.18,5830.0,10.0,5820.0,5890.0,5750.0,143015.0,095570
3,2021.11.19,5840.0,10.0,5740.0,5900.0,5720.0,54112.0,095570
4,2021.11.22,6040.0,200.0,5840.0,6110.0,5840.0,244193.0,095570
...,...,...,...,...,...,...,...,...
93714,2022.04.06,3095.0,85.0,3205.0,3260.0,2990.0,2719696.0,003280
93715,2022.04.07,2945.0,150.0,3070.0,3075.0,2940.0,965012.0,003280
93716,2022.04.08,3090.0,145.0,2955.0,3155.0,2890.0,2628593.0,003280
93717,2022.04.11,3020.0,70.0,3115.0,3170.0,3000.0,1004549.0,003280


In [189]:
stock_total['날짜'].unique()

array(['2021.11.16', '2021.11.17', '2021.11.18', '2021.11.19',
       '2021.11.22', '2021.11.23', '2021.11.24', '2021.11.25',
       '2021.11.26', '2021.11.29', '2021.11.30', '2021.12.01',
       '2021.12.02', '2021.12.03', '2021.12.06', '2021.12.07',
       '2021.12.08', '2021.12.09', '2021.12.10', '2021.12.13',
       '2021.12.14', '2021.12.15', '2021.12.16', '2021.12.17',
       '2021.12.20', '2021.12.21', '2021.12.22', '2021.12.23',
       '2021.12.24', '2021.12.27', '2021.12.28', '2021.12.29',
       '2021.12.30', '2022.01.03', '2022.01.04', '2022.01.05',
       '2022.01.06', '2022.01.07', '2022.01.10', '2022.01.11',
       '2022.01.12', '2022.01.13', '2022.01.14', '2022.01.17',
       '2022.01.18', '2022.01.19', '2022.01.20', '2022.01.21',
       '2022.01.24', '2022.01.25', '2022.01.26', '2022.01.27',
       '2022.01.28', '2022.02.03', '2022.02.04', '2022.02.07',
       '2022.02.08', '2022.02.09', '2022.02.10', '2022.02.11',
       '2022.02.14', '2022.02.15', '2022.02.16', '2022.

## 강사님 방법

In [190]:
def get_naver_stock(code, page=11):
    """
        code : 종목정보 
        page : 해당 페이지 정보 추출 
    """
    url = "https://finance.naver.com/item/sise_day.naver?code={}&page={}"
    head = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36 Edg/100.0.1185.36"}
    total = list()
    for x in range(1, page):
        r = requests.get(url.format(code, x), headers=head)
        total.append(pd.read_html(r.text)[0].dropna())
    total_df = pd.concat(total)
    total_df['종목코드'] = code
    return total_df.iloc[::-1]

In [173]:
def get_naver_stock2(code, page=11):
    """
        code : 종목정보 
        page : 해당 페이지 정보 추출 
    """
    url = "https://finance.naver.com/item/sise_day.naver?code={}&page={}"
    head = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36 Edg/100.0.1185.36"}
    
    total_df = pd.concat([pd.read_html(requests.get(url.format(code, x), headers=head).text)[0].dropna() for x in range(1, page)])
    total_df['종목코드'] = code
    return total_df.iloc[::-1]

In [ ]:
kospi_list = []
for idx, x in enumerate(krx_df.loc[kospi_mask, 'ISU_SRT_CD']):
    if idx % 100 == 0:
        print(idx)
    kosp_list.append(get_naver_stock(x,4))

## 2021년  워킹데이 추출

In [191]:
stock_df = get_naver_stock("005930", 50)

In [192]:
stock_df

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드
13,2020.04.17,51400.0,2400.0,50800.0,52000.0,50300.0,32041675.0,005930
12,2020.04.20,50100.0,1300.0,51400.0,51400.0,50000.0,21866354.0,005930
11,2020.04.21,49250.0,850.0,49400.0,49700.0,48700.0,27407543.0,005930
10,2020.04.22,49850.0,600.0,48700.0,50000.0,48350.0,18613864.0,005930
9,2020.04.23,49850.0,0.0,50200.0,50300.0,49500.0,18754442.0,005930
...,...,...,...,...,...,...,...,...
5,2022.04.06,68500.0,700.0,68600.0,68800.0,68500.0,15517308.0,005930
4,2022.04.07,68000.0,500.0,68500.0,68500.0,68000.0,20683327.0,005930
3,2022.04.08,67800.0,200.0,68100.0,68300.0,67700.0,15453191.0,005930
2,2022.04.11,67900.0,100.0,67800.0,68100.0,67400.0,12263735.0,005930


In [194]:
pd.to_datetime("2020.04.17")

Timestamp('2020-04-17 00:00:00')

In [197]:
stock_df['날짜'] = pd.to_datetime(stock_df['날짜'])

In [198]:
stock_df

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드
13,2020-04-17,51400.0,2400.0,50800.0,52000.0,50300.0,32041675.0,005930
12,2020-04-20,50100.0,1300.0,51400.0,51400.0,50000.0,21866354.0,005930
11,2020-04-21,49250.0,850.0,49400.0,49700.0,48700.0,27407543.0,005930
10,2020-04-22,49850.0,600.0,48700.0,50000.0,48350.0,18613864.0,005930
9,2020-04-23,49850.0,0.0,50200.0,50300.0,49500.0,18754442.0,005930
...,...,...,...,...,...,...,...,...
5,2022-04-06,68500.0,700.0,68600.0,68800.0,68500.0,15517308.0,005930
4,2022-04-07,68000.0,500.0,68500.0,68500.0,68000.0,20683327.0,005930
3,2022-04-08,67800.0,200.0,68100.0,68300.0,67700.0,15453191.0,005930
2,2022-04-11,67900.0,100.0,67800.0,68100.0,67400.0,12263735.0,005930


In [208]:
stock_df.loc[(stock_df['날짜'] > '2020-12-31') & (stock_df['날짜'] < '2022-01-01'), '날짜']

5    2021-01-04
4    2021-01-05
3    2021-01-06
2    2021-01-07
1    2021-01-08
        ...    
2    2021-12-24
1    2021-12-27
13   2021-12-28
12   2021-12-29
11   2021-12-30
Name: 날짜, Length: 248, dtype: datetime64[ns]

## 선수 연봉 내림차순 출력

In [210]:
# 연봉(만원)기준으로 내림차순 출력
df.sort_values(by=['연봉(만원)'], inplace = True, ascending=False)

In [211]:
df.head()

,선수명,등번호,생년월일,포지션,경력,입단 계약금,연봉,지명순위,입단년도,체중,신장,BMI,나이,탄생요일,연봉(만원),Team,용병
94,김광현,No.29,1988년 07월 22일,투수(좌투좌타),덕성초(안산리틀)-안산중앙중-안산공고-SK-세인트루이스,50000만원,810000만원,07 SK 1차,07SK,98,188,27.727478,35,금,810000.0,SSG 랜더스,False
164,추신수,No.17,1982년 07월 13일,외야수(좌투좌타),부산수영초-부산중-부산고-SK,NaN,270000만원,07 해외진출선수 특별지명,00시애틀,97,180,29.938272,41,화,270000.0,SSG 랜더스,False
175,구자욱,No.5,1993년 02월 12일,외야수(우투좌타),본리초-경복중-대구고-삼성-상무,13000만원,250000만원,12 삼성 2라운드 12순위,12삼성,75,189,20.996053,30,금,250000.0,삼성 라이온즈,False
170,한유섬,No.35,1989년 08월 09일,외야수(우투좌타),중앙초(해운대리틀)-대천중-경남고-경성대-SK-상무-SK,3000만원,240000만원,12 SK 9라운드 85순위,12SK,105,190,29.085873,34,수,240000.0,SSG 랜더스,False
439,나성범,No.47,1989년 10월 03일,외야수(좌투좌타),광주대성초-진흥중-진흥고-연세대-NC,30000만원,200000만원,12 NC 2라운드 10순위,12NC,100,183,29.860551,34,화,200000.0,KIA 타이거즈,False


## 팀별 연봉 1위 선수 출력

In [212]:
# 전체 선수 연봉 내림 차순 정렬 -> 팀별 그룹화 이후 가장 첫번째 선수 출력
df.groupby(['Team']).first()

,선수명,등번호,생년월일,포지션,경력,입단 계약금,연봉,지명순위,입단년도,체중,신장,BMI,나이,탄생요일,연봉(만원),용병
Team,,,,,,,,,,,,,,,,
KIA 타이거즈,나성범,No.47,1989년 10월 03일,외야수(좌투좌타),광주대성초-진흥중-진흥고-연세대-NC,30000만원,200000만원,12 NC 2라운드 10순위,12NC,100,183,29.860551,34,화,200000.0,False
KT 위즈,쿠에바스,No.32,1990년 10월 14일,투수(우투양타),베네수엘라 Universidad De Carabobo(대),250000달러,800000달러,19 KT 자유선발,19KT,98,188,27.727478,33,일,95892.0,True
LG 트윈스,김현수,No.22,1988년 01월 12일,외야수(우투좌타),쌍문초-신일중-신일고-두산-볼티모어-필라델피아,300000달러,150000만원,06 두산 육성선수,06두산,100,188,28.293345,35,화,150000.0,False
NC 다이노스,루친스키,No.40,1988년 12월 30일,투수(우투우타),미국 Tulsa Union(고)-미국 The Ohio State(대),300000달러,1600000달러,19 NC 자유선발,19NC,91,188,25.746944,35,금,191784.0,True
SSG 랜더스,김광현,No.29,1988년 07월 22일,투수(좌투좌타),덕성초(안산리틀)-안산중앙중-안산공고-SK-세인트루이스,50000만원,810000만원,07 SK 1차,07SK,98,188,27.727478,35,금,810000.0,False
두산 베어스,미란다,No.57,1989년 01월 10일,투수(좌투좌타),쿠바,150000달러,1600000달러,21 두산 자유선발,21두산,86,188,24.332277,34,화,191784.0,True
롯데 자이언츠,안치홍,No.13,1990년 07월 02일,내야수(우투우타),구지초(구리리틀)-대치중-서울고-KIA-경찰-KIA,18000만원,100000만원,09 KIA 2차 1라운드 1순위,09KIA,97,178,30.614821,33,월,100000.0,False
삼성 라이온즈,구자욱,No.5,1993년 02월 12일,외야수(우투좌타),본리초-경복중-대구고-삼성-상무,13000만원,250000만원,12 삼성 2라운드 12순위,12삼성,75,189,20.996053,30,금,250000.0,False
키움 히어로즈,요키시,No.43,1989년 07월 29일,투수(좌투우타),미국 Northwestern(대),300000달러,900000달러,19 키움 자유선발,19키움,93,188,26.312811,34,토,107878.5,True


In [220]:
# 각 팀별 연봉 2위 출력
df.groupby(['Team']).nth(1)

,선수명,등번호,생년월일,포지션,경력,입단 계약금,연봉,지명순위,입단년도,체중,신장,BMI,나이,탄생요일,연봉(만원),용병
Team,,,,,,,,,,,,,,,,
KIA 타이거즈,양현종,No.54,1988년 03월 01일,투수(좌투좌타),학강초-광주동성중-광주동성고-KIA-텍사스,20000만원,100000만원,07 KIA 2차 1라운드 1순위,07KIA,91,183,27.173102,35,화,100000.0,False
KT 위즈,데스파이네,No.40,1987년 04월 04일,투수(우투우타),미국 Hermanos Martinez Tamayo(고),300000달러,700000달러,20 KT 자유선발,20KT,92,182,27.774423,36,토,83905.5,True
LG 트윈스,켈리,No.3,1989년 10월 04일,투수(우투우타),미국 Sarasota(고),300000달러,900000달러,19 LG 자유선발,19LG,98,191,26.863299,34,수,107878.5,True
NC 다이노스,박건우,No.37,1990년 09월 08일,외야수(우투우타),역삼초-서울이수중-서울고-두산-경찰-두산,10000만원,190000만원,09 두산 2차 2라운드 10순위,09두산,80,184,23.629490,33,토,190000.0,False
SSG 랜더스,추신수,No.17,1982년 07월 13일,외야수(좌투좌타),부산수영초-부산중-부산고-SK,NaN,270000만원,07 해외진출선수 특별지명,00시애틀,97,180,29.938272,41,화,270000.0,False
두산 베어스,김재환,No.32,1988년 09월 22일,외야수(우투좌타),영랑초-상인천중-인천고-두산-상무,15000만원,150000만원,08 두산 2차 1라운드 4순위,08두산,90,183,26.874496,35,목,150000.0,False
롯데 자이언츠,이대호,No.10,1982년 06월 21일,내야수(우투우타),부산수영초-대동중-경남고-(영남사이버대)-롯데-시애틀,21000만원,80000만원,01 롯데 2차 1라운드 4순위,01롯데,130,194,34.541397,41,월,80000.0,False
삼성 라이온즈,오승환,No.21,1982년 07월 15일,투수(우투우타),도신초-우신중-경기고-단국대-삼성-한신-세인트루이스-토론토-콜로라도,18000만원,160000만원,05 삼성 2차 1라운드 5순위,05삼성,93,178,29.352355,41,목,160000.0,False
키움 히어로즈,푸이그,No.66,1990년 12월 07일,외야수(우투우타),쿠바 EIEFD(대),300000달러,700000달러,22 키움 자유선발,22키움,108,188,30.556813,33,금,83905.5,True
